## Glucocorticoid receptor - part 2 (MMP identification)

### Import libraries

In [ ]:
import pandas as pd

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import sys
sys.path.append('/home/daniel/wizepair2')
from mmpa.mmp import MMP

### Read in the dataset

In [ ]:
df = pd.read_csv('nr3c1_agonists_stripped.csv')
len(df.index)

### Create cartesian product of unique molecules tested in the same assay

In [ ]:
df_pairs = pd.merge(df, df, on=['target_pref_name', 'standard_type'])
df_pairs = df_pairs[['stripped_smiles_x', 'stripped_smiles_y']].drop_duplicates()
df_pairs

### Identify all pairs

In [ ]:
df_pairs = df_pairs.sample(frac=1).parallel_apply(lambda x: MMP(x.stripped_smiles_x, x.stripped_smiles_y, strictness=5, correspondence=1).execute(), axis=1)

In [ ]:
df_pairs = pd.json_normalize(df_pairs.explode())
df_pairs.sample(3).transpose()

### Drop failures and write output to file

In [ ]:
df_pairs = df_pairs[df_pairs.valid]
df_pairs.to_csv('nr3c1_agonists_pairs.csv', index=False)